In [1]:
from numpy.random import default_rng
import scanpy as sc
from anndata import AnnData
import scipy
sc.logging.print_header()
from sklearn.metrics.cluster import adjusted_rand_score
import numpy as np
import pandas as pd
import seaborn as sns
import os
import torch
import pandas as pd
from sklearn import metrics
import multiprocessing as mp
from sklearn.metrics.cluster import normalized_mutual_info_score
import pickle
import os.path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score

/home/vanhoan310/miniconda3/envs/gnn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.1 anndata==0.9.2 umap==0.5.7 numpy==1.24.4 scipy==1.8.1 pandas==1.4.2 scikit-learn==1.1.1 statsmodels==0.14.1 python-igraph==0.11.8 pynndescent==0.5.13


In [2]:
# scanpy==1.9.1 anndata==0.9.2 umap==0.5.7 numpy==1.24.4 scipy==1.8.1 pandas==1.4.2 scikit-learn==1.1.1 statsmodels==0.14.1
# python-igraph==0.11.8 pynndescent==0.5.13

## Read data

In [ ]:
# Read data
data_name = 'CBMCs_nCells_1002_nGenes_100_'
data_name_list = ['CBMCs_nCells_1002_nGenes_100_', 'CBMCs_nCells_1992_nGenes_1000_','CBMCs_nCells_5024_nGenes_100_', 'CBMCs_nCells_5024_nGenes_1000_']
RNA_counts = pd.read_csv('/data/hoan/spatial_transcriptomics/GraphBGM/Data/scDesign3/'+data_name +'RNA.csv', delimiter=',')
ADT_counts = pd.read_csv('/data/hoan/spatial_transcriptomics/GraphBGM/Data/scDesign3/'+data_name +'ADT.csv', delimiter=',')
metadata =pd.read_csv('/data/hoan/spatial_transcriptomics/GraphBGM/Data/scDesign3/'+data_name +'metadata.csv', delimiter=',') 
# Duplicate
RNA_counts = pd.concat([RNA_counts.copy().add_suffix(f'_{i+1}') for i in range(10)], axis=1)
# ADT_counts = pd.concat([ADT_counts.copy().add_suffix(f'_{i+1}') for i in range(10)], axis=1)

In [15]:
true_labels = metadata['label'].values

In [10]:
# Create anndata obj
adata_omics1 = AnnData(RNA_counts.values, obsm={"spatial": metadata.iloc[:,2:4].values}, uns={"spatial": metadata.iloc[:,2:4].values})
adata_omics1.obs_names = [f"Cell_{i:d}" for i in range(adata_omics1.n_obs)]
adata_omics1.var_names = list(RNA_counts.columns.values)
adata_omics2 = AnnData(ADT_counts.values, obsm={"spatial": metadata.iloc[:,2:4].values}, uns={"spatial": metadata.iloc[:,2:4].values})
adata_omics2.obs_names = [f"Cell_{i:d}" for i in range(adata_omics1.n_obs)]
adata_omics2.var_names = list(ADT_counts.columns.values)
adata_omics2.obsm['raw'] = adata_omics2.X

## Preprocessing data

In [11]:
n_features = adata_omics1.shape[1]
if n_features > 5000:
    sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)
sc.pp.scale(adata_omics1, zero_center=False, max_value=10)
if n_features > 5000:
    adata_omics1 =  adata_omics1[:, adata.var['highly_variable']]

In [12]:
## Perform PCA
sc.tl.pca(adata_omics1, n_comps=20)

## Run GraphGB and visualization

In [13]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils import runGraphBG
n_clusters = len(true_labels)
runGraphBG(adata_omics1, n_clusters=n_clusters, radius=50, key='X_pca', refinement=True, n_neighbors = 4, n_components = 20, covariance_type = 'tied', n_init = 5, max_iter = 1000)

tied ward
Data input shape n:  (1992, 20)


In [ ]:
GraphBG_RNA_labels = adata_omics1.obs['domain'].values
print(normalized_mutual_info_score(true_labels[true_labels>=0], GraphBG_RNA_labels[true_labels>=0]))

In [ ]:
runGraphBG(adata_omics1, n_clusters=n_clusters, radius=50, key='raw', refinement=True, n_neighbors = 4, n_components = 20, covariance_type = 'tied', n_init = 5, max_iter = 1000)

In [ ]:
GraphBG_ADT_labels = adata_omics2.obs['domain'].values
print(normalized_mutual_info_score(true_labels[true_labels>=0], GraphBG_ADT_labels[true_labels>=0]))

In [ ]:
output_df = pd.DataFrame({'GraphBG_RNA': GraphBG_RNA_labels, 'GraphBG_Protein': GraphBG_ADT_labels, 'ground_truth': true_labels})

In [ ]:
output_df

In [ ]:
output_df.to_csv('output/'+data_name+'_GraphBGUnimodal.csv', index = False)